<a href="https://colab.research.google.com/github/hehuan2363/CanadaInnovationProposal/blob/main/testQ%26AforCanadaPolicy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qU langchain openai
!pip install unstructured[local-inference]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install tiktoken

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 20.1 MB/s eta 0:00:00


In [3]:
from getpass import getpass

OPENAI_API_KEY=getpass("OpenAI API Key: ")


OpenAI API Key: ··········


In [4]:
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [5]:
# connect your Google Drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"

Mounted at /content/gdrive


In [ ]:
#loader = OnlinePDFLoader("https://wolfpaulus.com/wp-content/uploads/2017/05/field-guide-to-data-science.pdf")

In [6]:
#loader = UnstructuredPDFLoader(root_dir+ "field-guide-to-data-science.pdf")
from langchain.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://ised-isde.canada.ca/site/innovative-solutions-canada/en/digitization-and-cybersecurity")



In [7]:
data = loader.load()

In [8]:
print (f'You have {len(data)} document(s) in your data')
print (f'There are {len(data[0].page_content)} characters in your document')

You have 1 document(s) in your data
There are 48350 characters in your document


In [9]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)

In [10]:
texts = text_splitter.split_documents(data)

In [11]:
print (f'Now you have {len(texts)} documents')

Now you have 66 documents


Creat embedding now


In [12]:
!pip install pinecone-client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.2/177.2 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 26.1 MB/s eta 0:00:00


In [13]:
from langchain.vectorstores import Chroma, Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone

/usr/local/lib/python3.9/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [14]:
PINECONE_API_KEY = getpass("PINECONE_API_KEY: ")


PINECONE_API_KEY: ··········


In [15]:
PINECONE_API_ENV = 'us-east1-gcp'

In [16]:
# initialize pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_API_ENV  # next to api key in console
)
index_name = "testcanadapolicy"

In [17]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [18]:
docsearch = Pinecone.from_texts([t.page_content for t in texts], embeddings, index_name=index_name)

In [19]:
query = "What is the deadline for submiting the proposal?"
docs = docsearch.similarity_search(query, include_metadata=True)

In [20]:
docs

[Document(page_content='Bidders may submit multiple proposals, ensuring that each proposal is for a different Innovation.. The same or similar proposals may not be submitted twice for different problem statements. If the proposed innovation is similar or identical to an innovation previously submitted by the bidder that is currently active in a pool and has not been withdrawn prior to solicitation closing, the following assessment will be used to determine sufficient difference to proceed.', metadata={}),
 Document(page_content='SC2: Operational Readiness Validation\nThe Bidder must demonstrate that at the time of proposal submission, the proposed innovation is ready for testing in an operational environment, i.e. at minimum TRL 7\xa0per the ISC Technology Readiness Level Scale.\n\n\n\nPass\n\n\nThe Bidder has provided evidence demonstrating that, at minimum, the proposed innovation is a functional prototype (form, fit, and function) ready for demonstration in an appropriate operationa

In [21]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

In [22]:
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)
chain = load_qa_chain(llm, chain_type="stuff")

In [23]:
query = "What is the deadline for submiting the proposal?"
docs = docsearch.similarity_search(query, include_metadata=True)

In [24]:
chain.run(input_documents=docs, question=query)

" I don't know."

In [25]:
query = "Who can submit the proposal?"
docs = docsearch.similarity_search(query, include_metadata=True)

In [26]:
chain.run(input_documents=docs, question=query)

' A Canadian person or entity submitting a proposal on its own behalf and having a place of business in Canada where the person or entity conducts activities on a permanent basis that is clearly identified by name and accessible during normal working hours.'

In [27]:
query = "How can I increase my chance to win for this program?"
docs = docsearch.similarity_search(query, include_metadata=True)
chain.run(input_documents=docs, question=query)

' You can increase your chance to win by ensuring that you provide detailed information that meets the criteria outlined in the program. Make sure that you demonstrate that you have sufficient secured funds and a credible financial strategy to successfully commercialize the proposed innovation, that you have identified economic benefits to the innovation ecosystem in Canada, and that you have implemented all measures to achieve and maintain diversity, inclusivity and gender equity. Additionally, you should provide a credible commercialization strategy, market risk strategies and mitigation strategies that could support target market entry.'

In [28]:
query = "how much money could I get if my proposal get selected"
docs = docsearch.similarity_search(query, include_metadata=True)
chain.run(input_documents=docs, question=query)

' If your proposal is selected, you could receive up to $1,100,000 CAD in the Standard Component or $2,300,000 CAD in the Military Component, not including applicable taxes, shipping costs, and travel and living expenses, where applicable.'